# Time-series Generative Adversarial Network (TimeGAN)

## Imports & Settings

Adapted from the excellent paper by Jinsung Yoon, Daniel Jarrett, and Mihaela van der Schaar: \\
Time-series Generative Adversarial Networks, \\
Neural Information Processing Systems (NeurIPS), 2019.

- Last updated Date: April 24th 2020
- Original code author: Jinsung Yoon (jsyoon0823@gmail.com)

In [ ]:
!pip install gdown
!gdown --id 1UgRteM_oc10aI1Eb0co8BRIUKGHXPFId

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Mount if stock data stored in drive

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from pathlib import Path
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

!pip install torchinfo
import torchinfo

import matplotlib.pyplot as plt
import seaborn as sns

# for downloading and storing data
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
sns.set_style('white')

## Download and store data

### Set data store path

In [ ]:
DATA_STORE = Path('assets.h5')

### Quandl Wiki Prices

Quandl announced the end of community support (updates). The historical data are useful as a first step towards demonstrating the application of the machine learning solutions, just ensure you design and test your own algorithms using current, professional-grade data.


1. Follow the instructions to create a free [NASDAQ account](https://data.nasdaq.com/sign-up)
2. [Download](Download) the entire WIKI/PRICES data
3. Extract the .zip file,
4. Move to this directory and rename to wiki_prices.csv
5. Run the below code to store in fast HDF format (see Chapter 02 on Market & Fundamental Data for details).


Alternatively, instead of all of that, download [this file](https://drive.google.com/file/d/1UgRteM_oc10aI1Eb0co8BRIUKGHXPFId/view?usp=share_link)

In [ ]:
# SomethingChanged
# wiki_prices_concat
file_path = 'wiki_prices.csv'
# if not(os.path.exists(file_path)) and os.path.exists(f'drive/MyDrive/{file_path}'):
#   file_path = f'drive/MyDrive/{file_path}'

df = (pd.read_csv(file_path,
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())

print(df.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)

## Experiment Path

In [ ]:
results_path = Path('time_gan')
if not results_path.exists():
    results_path.mkdir()

In [ ]:
experiment = 4

In [ ]:
log_dir = results_path / f'experiment_{experiment:02}'
if not log_dir.exists():
    log_dir.mkdir(parents=True)

In [ ]:
hdf_store = results_path / 'TimeSeriesGAN.h5'

## Prepare data

### Parameters

In [ ]:
seq_len = 24
n_seq = 6
batch_size = 128

# torch specific
random_seed = 42
torch.manual_seed(random_seed)

AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS=int(os.cpu_count() / 2)

In [ ]:
tickers = ['BA', 'CAT', 'DIS', 'GE', 'IBM', 'KO']

In [ ]:
def select_data():
    df = (pd.read_hdf('assets.h5', 'quandl/wiki/prices')
          .adj_close
          .unstack('ticker')
          .loc['2000':, tickers]
          .dropna())
    df.to_hdf(hdf_store, 'data/real')

In [ ]:
select_data()

### Plot Series

In [ ]:
df = pd.read_hdf(hdf_store, 'data/real')
axes = df.div(df.iloc[0]).plot(subplots=True,
                               figsize=(14, 6),
                               layout=(3, 2),
                               title=tickers,
                               legend=False,
                               rot=0,
                               lw=1,
                               color='k')
for ax in axes.flatten():
    ax.set_xlabel('')

plt.suptitle('Normalized Price Series')
plt.gcf().tight_layout()
sns.despine();

### Correlation

In [ ]:
sns.clustermap(
    df.corr(),
    annot=True,
    fmt='.2f',
    cmap=sns.diverging_palette(h_neg=20, h_pos=220),
    center=0
)

### Normalise data

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df).astype(np.float32)

### Create rolling window sequences

In [ ]:
data = np.array([scaled_data[i:i + seq_len] for i in range(len(df) - seq_len)])
n_windows = len(data)

### Create torch.utils.data.DataLoader

Do not need buffer_size and repeat() in pytorch as Pytorch will shuffle the data internally.

Can set the number of epochs while creating the DataLoader which will repeat the dataset for specified number of times.

In [ ]:
real_series = torch.utils.data.DataLoader(
               torch.from_numpy(data),
               shuffle=True,
               batch_size=batch_size,
               num_workers=NUM_WORKERS,
               drop_last=True
               )
real_series_iter = iter(real_series)

from itertools import cycle
real_series_iter = cycle(real_series_iter)

### Set up random series generator

In [ ]:
def make_random_data():
  while True:
    yield np.random.uniform(low=0, high=1, size=(seq_len, n_seq))

!editpls

We use the Python generator to feed a tf.data.Dataset that continues to call the random number generator as long as necessary and produces the desired batch size

In [ ]:
train_steps = 10000
gamma = 1

In [ ]:
def make_random_data():
    while True:
        yield torch.rand((batch_size, seq_len, n_seq))

random_series = make_random_data()

## TimeGAN Components

### Network parameters

In [ ]:
hidden_dim = 24
num_layers = 3

### Set up logger

You can use the `add_scalar()`, `add_histogram()`, `add_image()` etc methods of the SummaryWriter class to write the various types of data to the tensorboard.

You can also use `with SummaryWriter(log_dir) as writer`: to automatically close the writer once you are done with it.

Also, you could use `log_dir.resolve()` instead of `log_dir.as_posix()` in Pytorch.

In [ ]:
writer = SummaryWriter(log_dir.resolve())

### RNN block generator


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(RNN, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.fc(x)
        return torch.sigmoid(x)

###  Embedder & Recovery

In [ ]:
embedder = RNN(n_seq, hidden_dim, hidden_dim).to(device)
recovery = RNN(hidden_dim, hidden_dim, n_seq).to(device)

In [ ]:
input_shape = (batch_size, seq_len, n_seq) # torch.Size([128, 24, 6])
dummy_input = torch.randn(input_shape).to(device)

embedded_data = embedder(dummy_input)
recovered_data = recovery(embedded_data)
embedded_data.shape, recovered_data.shape

### Generator & Discriminator

It's important to note that the supervisor network is not mandatory to use in TimeGAN and its usage might be dependent on the specific dataset and problem you are trying to solve.

In [ ]:
generator = RNN(n_seq, hidden_dim, hidden_dim).to(device)
discriminator = RNN(hidden_dim, hidden_dim, 1).to(device)
supervisor = RNN(hidden_dim, hidden_dim, hidden_dim, 2).to(device)

## TimeGAN Training

### Settings

Settings have been done when setting the random series generator

### Generic Loss Functions

In [ ]:
mse = nn.MSELoss()
bce = nn.BCELoss()

# Phase 1: Autoencoder Training

An autoencoder is a neural network architecture that is trained to reconstruct its input. It consists of two main components: an encoder and a decoder. The encoder maps the input data to a lower-dimensional representation (also known as the bottleneck or latent representation), and the decoder maps the bottleneck representation back to the original input space. The goal of the autoencoder is to learn a compact and informative representation of the input data that can be used for various tasks, such as data compression, denoising, and anomaly detection.

In the given notebook, the autoencoder architecture is composed of an `embedder` model and a `recovery` model. The `embedder` model is responsible for encoding the input data and the `recovery` model is responsible for decoding the encoded data. The `embedder` and `recovery` models are combined to form the autoencoder, where the output of the `embedder` model is passed as input to the `recovery` model.

The `embedder` model is a one-layer LSTM model with a hidden size of 24 and the `recovery` model is also a one-layer LSTM model, but with a hidden size of 6. The `embedder` maps the input data to a lower-dimensional representation with 24 dimensions, and the `recovery` maps this representation back to the original input space with 6 dimensions.

The autoencoder is trained using the Mean Squared Error (MSE) loss function, which measures the difference between the input and the reconstructed output. The goal of the training is to minimize this MSE loss and learn a good representation of the input data.

The output of the autoencoder (`x_tilde`) is then used as input for the TimeGAN, which is an extension of the GAN architecture that is designed to generate synthetic time series data. The TimeGAN architecture is composed of a generator, a discriminator, and a supervisor. The generator generates synthetic data, the discriminator discriminates between real and synthetic data, and the supervisor guides the generator to produce realistic synthetic data.


## Architecture

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, embedder, recovery):
        # super(Autoencoder, self).__init__()
        super().__init__()
        self.embedder = embedder
        self.recovery = recovery

        # self.embedder =

    def forward(self, X):
        H = self.embedder(X) # embedded_data
        X_tilde = self.recovery(H) # recovered_data
        return X_tilde

autoencoder = Autoencoder(embedder, recovery)

In [ ]:
print(autoencoder)

## Autoencoder Optimizer

In [ ]:
autoencoder_optimizer = optim.Adam(autoencoder.parameters()) #lr, weight_decay could be specified here

## Autoencoder Training Step

In [ ]:
def train_autoencoder_init(x):
    x_tilde = autoencoder(x)
    loss = mse(x, x_tilde)
    autoencoder_loss = 10 * torch.sqrt(loss)
    autoencoder_loss.backward()
    autoencoder_optimizer.step()
    autoencoder_optimizer.zero_grad()
    return torch.sqrt(loss)

## Autoencoder Training Loop

In [ ]:
for step in tqdm(range(train_steps)):
    X_ = next(real_series_iter).to(device)
    step_e_loss_t0 = train_autoencoder_init(X_)
    writer.add_scalar('Loss Autoencoder Init', step_e_loss_t0, step)

## Persist model

In [ ]:
torch.save(autoencoder, log_dir / 'autoencoder')
autoencoder = torch.load(log_dir / 'autoencoder').to(device)
autoencoder.eval()

# Phase 2: Supervised Training

## Define Optimiser

In [ ]:
supervisor_optimizer = optim.Adam(autoencoder.parameters())

## Train Step

In this code, `train_supervisor` is defined as a function that takes in input `x`. The `embedder` and `supervisor` models are used to compute the `h` and `h_hat_supervised` respectively. The Mean Squared Error loss is calculated between the `h[:, 1:, :]` and `h_hat_supervised[:, :-1, :]`.
Then the optimizer is used to update the parameters of the `supervisor` model. `optimizer.step()` performs a single optimization step (parameter update). `optimizer.zero_grad()` is used to clear the gradients of all optimized torch.Tensor s.
Note that in PyTorch there is no need to use `tf.function` and `tf.GradientTape` . Instead, Pytorch uses autograd for computing gradients and `backward()` for backpropagation.

In [ ]:
def train_supervisor(x):
    x.requiresGrad = True
    h = embedder(x)
    h_hat_supervised = supervisor(h)
    g_loss_s = nn.MSELoss()(h[:, 1:, :], h_hat_supervised[:, :-1, :])
    g_loss_s.backward()
    supervisor_optimizer.step()
    supervisor_optimizer.zero_grad()
    return g_loss_s

## Training Loop

In [ ]:
for step in tqdm(range(train_steps)):
    X_ = next(real_series_iter).to(device)
    step_g_loss_s = train_supervisor(X_)
    writer.add_scalar('Loss Generator Supervised Init', step_g_loss_s, step)

## Persist model

In [ ]:
torch.save(supervisor, log_dir / 'supervisor')
supervisor = torch.load(log_dir / 'supervisor').to(device)
supervisor.eval()

# Joint Training

## Generator

### Adversarial Architecture - Supervised

In [ ]:
Z = torch.randn((batch_size, seq_len, n_seq)).to(device)
E_hat = generator(Z)
H_hat = supervisor(E_hat)
Y_fake = discriminator(H_hat)

modules = nn.ModuleList([generator, supervisor, discriminator])
adversarial_supervised = nn.Sequential(*modules)
adversarial_supervised.name = 'AdversarialNetSupervised'

In [ ]:
print(adversarial_supervised)

### Adversarial Architecture in Latent Space

In [ ]:
class AdversarialNet(nn.Module):
    def __init__(self, generator, discriminator):
        super(AdversarialNet, self).__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, x):
        e_hat = self.generator(x)
        y_fake_e = self.discriminator(e_hat)
        return y_fake_e

adversarial_emb = AdversarialNet(generator, discriminator).to(device)
adversarial_emb.name = 'AdversarialNet'

In [ ]:
print(adversarial_emb)

### Mean & Variance Loss

In [ ]:
class SyntheticData(nn.Module):
    def __init__(self, generator, supervisor, recovery):
        super(SyntheticData, self).__init__()
        self.generator = generator
        self.supervisor = supervisor
        self.recovery = recovery

    def forward(self, x):
        e_hat = self.generator(x)
        h_hat = self.supervisor(e_hat)
        x_hat = self.recovery(h_hat)
        return x_hat

synthetic_data = SyntheticData(generator, supervisor, recovery).to(device)
synthetic_data.name = 'SyntheticData'

In [ ]:
print(synthetic_data)

In [ ]:
def get_generator_moment_loss(y_true, y_pred):
    y_true_mean = torch.mean(y_true, dim=0)
    y_true_var = torch.var(y_true, dim=0) + 1e-6
    y_pred_mean = torch.mean(y_pred, dim=0)
    y_pred_var = torch.var(y_pred, dim=0) + 1e-6
    g_loss_mean = torch.mean(torch.abs(y_true_mean - y_pred_mean))
    g_loss_var = torch.mean(torch.abs(torch.sqrt(y_true_var) - torch.sqrt(y_pred_var)))
    return g_loss_mean + g_loss_var

## Discriminator

### Architecture: Real Data

In [ ]:
discriminator_model = nn.Sequential(embedder, discriminator).to(device)

In [ ]:
print(discriminator_model)

## Optimisers

In [ ]:
generator_optimizer = optim.Adam(generator.parameters())
discriminator_optimizer = optim.Adam(discriminator.parameters())
embedding_optimizer = optim.Adam(embedder.parameters())

## Generator Train Step

In [ ]:
def train_generator(x, z):
    # Calculate generator losses
    y_fake = adversarial_supervised(z)
    generator_loss_unsupervised = bce(y_fake, torch.ones_like(y_fake))

    y_fake_e = adversarial_emb(z)
    generator_loss_unsupervised_e = bce(y_fake_e, torch.ones_like(y_fake_e))

    h = embedder(x)
    h_hat_supervised = supervisor(h)
    generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

    x_hat = synthetic_data(z)
    generator_moment_loss = get_generator_moment_loss(x, x_hat)

    generator_loss = (generator_loss_unsupervised +
                      generator_loss_unsupervised_e +
                      100 * torch.sqrt(generator_loss_supervised) +
                      100 * generator_moment_loss)

    # Calculate gradients and update generator and supervisor parameters
    generator.zero_grad()
    supervisor.zero_grad()
    generator_loss.backward()
    generator_optimizer.step()
    supervisor_optimizer.step()
    return generator_loss_unsupervised, generator_loss_supervised, generator_moment_loss


## Embedding Train Step

In [ ]:
def train_embedder(x):
    h = embedder(x)
    h_hat_supervised = supervisor(h)
    generator_loss_supervised = nn.MSELoss()(h[:, 1:, :], h_hat_supervised[:, 1:, :])

    x_tilde = autoencoder(x)
    embedding_loss_t0 = nn.MSELoss()(x, x_tilde)
    e_loss = 10 * torch.sqrt(embedding_loss_t0) + 0.1 * generator_loss_supervised

    optimizer = torch.optim.Adam(list(embedder.parameters()) + list(recovery.parameters()), lr=0.001)
    optimizer.zero_grad()
    e_loss.backward()
    optimizer.step()
    return torch.sqrt(embedding_loss_t0)

## Discriminator Train Step

In [ ]:
def get_discriminator_loss(x, z):
    y_real = discriminator_model(x)
    discriminator_loss_real = nn.BCELoss()(y_real, torch.ones_like(y_real))

    y_fake = adversarial_supervised(z)
    discriminator_loss_fake = nn.BCELoss()(y_fake, torch.zeros_like(y_fake))

    y_fake_e = adversarial_emb(z)
    discriminator_loss_fake_e = nn.BCELoss()(y_fake_e, torch.zeros_like(y_fake_e))
    return discriminator_loss_real + discriminator_loss_fake + gamma * discriminator_loss_fake_e

In [ ]:
def train_discriminator(x, z):
    discriminator_loss = get_discriminator_loss(x, z)

    discriminator_optimizer.zero_grad()
    discriminator_loss.backward()
    discriminator_optimizer.step()
    return discriminator_loss

## Training Loop

In [ ]:
step_g_loss_u = step_g_loss_s = step_g_loss_v = step_e_loss_t0 = step_d_loss = 0
autoencoder.train()
supervisor.train()
for step in range(train_steps):
    # Train generator (twice as often as discriminator)
    for kk in range(2):
        X_ = next(real_series_iter).to(device)
        Z_ = next(random_series).to(device)

        # Train generator
        step_g_loss_u, step_g_loss_s, step_g_loss_v = train_generator(X_, Z_)
        # Train embedder
        step_e_loss_t0 = train_embedder(X_)

    X_ = next(real_series_iter).to(device)
    Z_ = next(random_series).to(device)
    step_d_loss = get_discriminator_loss(X_, Z_)
    if step_d_loss > 0.15:
        step_d_loss = train_discriminator(X_, Z_)

    if step % 1000 == 0:
        print(f'{step:6,.0f} | d_loss: {step_d_loss:6.4f} | g_loss_u: {step_g_loss_u:6.4f} | '
              f'g_loss_s: {step_g_loss_s:6.4f} | g_loss_v: {step_g_loss_v:6.4f} | e_loss_t0: {step_e_loss_t0:6.4f}')


## Persist Synthetic Data Generator


In [ ]:
torch.save(synthetic_data, log_dir / 'synthetic_data') # uncomment if not using saved model files
synthetic_data = torch.load(log_dir / 'synthetic_data')
synthetic_data.eval()

# Generate Synthetic Data

In [ ]:
generated_data = []
for i in range(int(n_windows / batch_size)):
    Z_ = next(random_series).to(device)
    d = synthetic_data(Z_)
    generated_data.append(d)

In [ ]:
len(generated_data)

In [ ]:
generated_data = np.array(np.vstack([d.detach().cpu().numpy() for d in generated_data]))
generated_data.shape

In [ ]:
np.save(log_dir / 'generated_data.npy', generated_data)

## Rescale

In [ ]:
generated_data = (scaler.inverse_transform(generated_data
                                           .reshape(-1, n_seq))
                  .reshape(-1, seq_len, n_seq))
generated_data.shape

## Persist Data

In [ ]:
with pd.HDFStore(hdf_store) as store:
    store.put('data/synthetic', pd.DataFrame(generated_data.reshape(-1, n_seq),
                                             columns=tickers))

## Plot Sample Series

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14, 7))
axes = axes.flatten()

index = list(range(1, 25))
synthetic = generated_data[np.random.randint(n_windows)]

idx = np.random.randint(len(df) - seq_len)
real = df.iloc[idx: idx + seq_len]

for j, ticker in enumerate(tickers):
    (pd.DataFrame({'Real': real.iloc[:, j].values,
                   'Synthetic': synthetic[:, j]})
     .plot(ax=axes[j],
           title=ticker,
           secondary_y='Synthetic', style=['-', '--'],
           lw=1))
sns.despine()
fig.tight_layout()

# Visualize Real and Synthetic Data

## Load Data

In [ ]:
path = Path('time_gan')

In [ ]:
def get_real_data():
    df = pd.read_hdf(hdf_store, 'data/real').sort_index()

    # Preprocess the dataset:
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    data = []
    for i in range(len(df) - seq_len):
        data.append(scaled_data[i:i + seq_len])
    return data


real_data = get_real_data()

In [ ]:
n = len(real_data)

In [ ]:
np.asarray(real_data).shape

In [ ]:
synthetic_data = np.load(path / f'experiment_{experiment:02d}' / 'generated_data.npy')
synthetic_data.shape

In [ ]:
real_data = real_data[:synthetic_data.shape[0]]

## Prepare Sample

In [ ]:
sample_size = 250
idx = np.random.permutation(len(real_data))[:sample_size]

In [ ]:
# Data preprocessing
real_sample = np.asarray(real_data)[idx]
synthetic_sample = np.asarray(synthetic_data)[idx]

In [ ]:
real_sample_2d = real_sample.reshape(-1, seq_len)
synthetic_sample_2d = synthetic_sample.reshape(-1, seq_len)

In [ ]:
real_sample_2d.shape, synthetic_sample_2d.shape

## Visualization in 2D: A Qualitative Assessment of Diversity

### Run PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(real_sample_2d)
pca_real = (pd.DataFrame(pca.transform(real_sample_2d))
            .assign(Data='Real'))
pca_synthetic = (pd.DataFrame(pca.transform(synthetic_sample_2d))
                 .assign(Data='Synthetic'))
pca_result = pca_real.append(pca_synthetic).rename(
    columns={0: '1st Component', 1: '2nd Component'})

### Plot t-SNE

In [ ]:
tsne_data = np.concatenate((real_sample_2d,
                            synthetic_sample_2d), axis=0)

tsne = TSNE(n_components=2,
            verbose=1,
            perplexity=40)
tsne_result = tsne.fit_transform(tsne_data)

In [ ]:
tsne_result = pd.DataFrame(tsne_result, columns=['X', 'Y']).assign(Data='Real')
tsne_result.loc[sample_size*6:, 'Data'] = 'Synthetic'

### Plot result

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14, 5))

sns.scatterplot(x='1st Component', y='2nd Component', data=pca_result,
                hue='Data', style='Data', ax=axes[0])
sns.despine()
axes[0].set_title('PCA Result')


sns.scatterplot(x='X', y='Y',
                data=tsne_result,
                hue='Data',
                style='Data',
                ax=axes[1])
sns.despine()
for i in [0, 1]:
    axes[i].set_xticks([])
    axes[i].set_yticks([])

axes[1].set_title('t-SNE Result')
fig.suptitle('Assessing Diversity: Qualitative Comparison of Real and Synthetic Data Distributions',
             fontsize=14)
fig.tight_layout()
fig.subplots_adjust(top=.88);

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
download_file('/kaggle/working/time_gan', 'out2')